In [6]:
# Import Dependencies
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urljoin
import concurrent.futures
import sys
import pandas as pd
import json
import queue
import threading

In [7]:
def crawl_web(start, success_file_path, error_file_path):
    unseen = set([start])
    seen = set([])

    with concurrent.futures.ThreadPoolExecutor(max_workers=6) as executor:
        while unseen:
            url = unseen.pop()
            seen.add(url)
            executor.submit(crawl_website, url, seen, unseen, success_file_path, error_file_path)

    print(f"Finished crawling {start}")
    return seen 

def crawl_website(url, seen, unseen, success_file_path, error_file_path):
    try:
        new_links = getNewLinks(url, seen, unseen, success_file_path)
        with concurrent.futures.ThreadPoolExecutor(max_workers=6) as executor:
            for link in new_links:
                print(f"Adding: {link}")
                unseen.add(link)  
                executor.submit(crawl_website, link, seen, unseen, success_file_path, error_file_path)
    except:
        print(f"Error crawling {url} due to {sys.exc_info()[0]}")
        with open(error_file_path, 'a') as f:
            f.write(f"{url}\n")

def save_links_to_file(links, file_path):
    with open(file_path, 'a') as f:
        for link in links:
            if not is_duplicate_link(link, file_path):
                f.write(f"{link}\n")
                
def getNewLinks(url, seen, unseen, success_file_path):
    response = requests.get(url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        links = [link['href'] for link in soup.find_all('a') if link.has_attr('href') and not any(ext in link['href'] for ext in ['.png', '.pdf', '.jpg', '.jpeg', '~json/', 'javascript:;', 'mailto:', 'webcal:', 'email-protection', 'catalyst'])]
        links = [urljoin(url, link) for link in links]
        links = [link for link in links if link not in seen and link not in unseen and urlparse(link).netloc.endswith('library.jhu.edu')] # Change this to limit domain
        save_links_to_file(links, success_file_path)
        return links
    else:
        print(f"Error getting {url} with status code {response.status_code} and reason {response.reason}")
        print(f"Error full response: {response.text}")
    return []

def is_duplicate_link(link, file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            if link.strip() == line.strip():
                return True
    return False

def domain(url1, url2):
    return urlparse(url1).netloc == urlparse(url2).netloc

In [8]:
# crawled_links = crawl_web('https://library.jhu.edu/', "../backend/data.nosync/library-jhu-edu/success_links.txt", "../backend/data.nosync/library-jhu-edu/error_links.txt")

In [9]:
# Load crawled links from file into a dataframe (in case of termination)
df = pd.read_csv("../backend/data.nosync/library-jhu-edu/success_links.txt", header=None, error_bad_lines=False)

# Rename column
df.rename(columns={0: "Link"}, inplace=True)

# Normalize dataframe (remove any links that are not jhu.edu in base url or no https:// in the link)
df = df[df['Link'].str.contains("library.jhu.edu")]
df


/var/folders/ds/33bb8_h50z75_0klrf43_bqm0000gn/T/ipykernel_16458/588397982.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv("../backend/data.nosync/library-jhu-edu/success_links.txt", header=None, error_bad_lines=False)
Skipping line 24580: expected 1 fields, saw 2
Skipping line 32875: expected 1 fields, saw 2
Skipping line 33450: expected 1 fields, saw 2
Skipping line 50838: expected 1 fields, saw 2



,Link
0,https://catalyst.library.jhu.edu/shibboleth_lo...
1,https://catalyst.library.jhu.edu/shibboleth_lo...
2,https://catalyst.library.jhu.edu/shibboleth_lo...
3,https://catalyst.library.jhu.edu/shibboleth_lo...
4,https://catalyst.library.jhu.edu/shibboleth_lo...
...,...
63351,http://jhir.library.jhu.edu/handle/1774.2/836/...
63352,http://jhir.library.jhu.edu/handle/1774.2/836/...
63353,http://jhir.library.jhu.edu/handle/1774.2/836/...
63354,http://jhir.library.jhu.edu/handle/1774.2/836/...


In [10]:
def clean_text(text):
    # Remove leading/trailing whitespace, condense multiple whitespaces to single
    return ' '.join(text.split())

def scrape_page(url, q):
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')

    tags = ['p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'span', 'li', 'div', 'table', 'th', 'td', 'tr', 'ol', 'ul', 'blockquote', 'pre', 'code', 'caption', 'dt', 'dd']

    text_data = []
    for tag in tags:
        elements = soup.find_all(tag)
        for element in elements:
            text = clean_text(element.get_text())
            if text:
                text_data.append(text)

    json_data = []
    for i in range(len(text_data) - 1):
        json_data.append({
            'prompt': text_data[i],
            'completion': text_data[i + 1]
        })
        
    # add to queue
    q.put(json_data)
    
def write_to_file(q):
    with open('../backend/data.nosync/library-jhu-edu/data.json', 'a') as f:
        while True:
            data = q.get()
            if data is None:
                break
            json.dump(data, f)
            f.write('\n')
            q.task_done()

In [11]:
# List of URLs to scrape
urls = df['Link'].tolist()

# create a queue
q = queue.Queue()

# create a separate thread to write data to file
file_writer = threading.Thread(target=write_to_file, args=(q,))
file_writer.start()

with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    futures = {executor.submit(scrape_page, url, q) for url in urls}
    concurrent.futures.wait(futures)

# stop the file writing thread
q.put(None)
file_writer.join()


/var/folders/ds/33bb8_h50z75_0klrf43_bqm0000gn/T/ipykernel_16458/1453435867.py:7: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(res.text, 'html.parser')
/Users/benjamin/anaconda3/lib/python3.10/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [12]:

# Load pairs into a dataframe
with open('../backend/data.nosync/library-jhu-edu/data.json', 'r') as f:
    data = []
    for line in f:
        # Error handling
        try:
            data.extend(json.loads(line))
        except:
            pass
        
pc_pairs_df = pd.DataFrame(data)

print(f"Number of Prompt/Completion pairs before cleaning: {len(pc_pairs_df)}")
print(f"Number of words scraped (prompt): {pc_pairs_df['prompt'].str.split().str.len().sum()}")
print(f"Number of words scraped (completion): {pc_pairs_df['completion'].str.split().str.len().sum()}")

# Remove any prompts/completion pairs that contains characters less than 15
pc_pairs_df = pc_pairs_df[pc_pairs_df['prompt'].str.len() > 15]
pc_pairs_df = pc_pairs_df[pc_pairs_df['completion'].str.len() > 15]

# Remove duplicates
pc_pairs_df.drop_duplicates(subset='prompt', keep='last', inplace=True)

# Save to JSON file in the format of {"prompt": "prompt text", "completion": "completion text"}
pc_pairs_df.to_json('../backend/data.nosync/library-jhu-edu/prompt-completion-pairs.json', orient='records', lines=True)

print(f"Number of Prompt/Completion pairs after cleaning: {len(pc_pairs_df)}")

pc_pairs_df

Number of Prompt/Completion pairs before cleaning: 8830924
Number of words scraped (prompt): 221535996
Number of words scraped (completion): 225531440
Number of Prompt/Completion pairs after cleaning: 342748


,prompt,completion
115,1997 – latest: SpringerLINK Contemporary 1997-...,1997 – latest: SpringerLink Journals - AutoHol...
123,Find It @ Johns Hopkins Libraries Short link J...,Find It @ Johns Hopkins Libraries Short link
127,Journal e & i Elektrotechnik und Informationst...,Journal e & i Elektrotechnik und Informationst...
128,Journal e & i Elektrotechnik und Informationst...,Journal e & i Elektrotechnik und Informationst...
130,Journal e & i Elektrotechnik und Informationst...,Online Access 1997 – latest: SpringerLINK Cont...
...,...,...
8830911,War Powers Resolution of 1973 [1],war termination [1]
8830912,war termination [1],war-fighting [1]
8830913,war-fighting [1],Warburg effect [1]
8830922,"JScholarship Home Theses and Dissertations, El...",0-9 A B C D E F G H I J K L M N O P Q R S T U ...


In [13]:
# Get dataset statistics
# Amount of links (from links.txt)
print(f"Number of links: {len(df)}")

Number of links: 63356
